# Analyse the literature

## Imports and function definitions

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
# NOTE: These two lines let us preview the plots in a notebook, but the inline
# plots will have the *wrong font* ):
from matplotlib.backends.backend_pgf import FigureCanvasPgf
mpl.backend_bases.register_backend('pdf', FigureCanvasPgf)
import pandas as pd
import numpy as np

plt.rcParams.update({
    'font.family': 'Cambria',
    'text.usetex': True,
    "pgf.rcfonts": False,
    'pgf.texsystem': 'xelatex',
    'pgf.preamble': '\n'.join([
        r'\usepackage[math-style=TeX,bold-style=TeX]{unicode-math}',
        r'\setmainfont{Cambria}',
        r'\setmathfont{Cambria Math}',
    ])
})
ORAL_DIR = 'oral/imgs'

In [ ]:
import pandas as pd
# import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
# No touchie. Make the font the same as the LaTeX paper
# mpl.rc('font', family='serif', serif='cmr10', size=18)
# mpl.rc('axes.formatter', use_mathtext=True)
# import matplotlib.pyplot as plt
import seaborn as sns
import re
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 30)
import numpy as np

A4_WIDTH_INCHES = 8.25
WIDTH = 5.5

FINAL = True

def watermark(ax):
    if FINAL:
        return
    ax.text(
        0.5, 
        0.5, 
        'Visualisation not final', 
        transform=ax.transAxes,
        fontsize=40, 
        color='gray', 
        alpha=0.5,
        ha='center', 
        va='center', 
        rotation=30
    )

    
replacements = {
    "5Dt": "5DT",
    "App": "Application",
    "Ar": "Augmented Reality",
    "Based On": "Based-on",
    "Cnn": "CNN",
    "Cyberglove": "CyberGlove",
    "Cyberglove Ii": "CyberGlove II",
    "Dataglove": "DataGlove",
    "Deep Q Network": "Deep Q-Network",
    "Emg": "EMG",
    "Ffnn": "FFNN",
    "Gmm": "GMM",
    "Gru": "GRU",
    "Hmm": "HMM",
    "Imu": "IMU",
    "K Means": "k-Means",
    "Knn": "KNN",
    "Lstm": "LSTM",
    "Msc": "Master's Thesis",
    "Myomonitor Iv": "Myomonitor IV",
    "Nn": "NN",
    "Pca": "PCA",
    "Powerglove": "PowerGlove",
    "Rgb": "RGB",
    "Rgbd": "RGBD",
    "Rnn": "RNN",
    "Sl": "Sign Language",
    "Som": "SOM",
    "Svd": "Svd",
    "Svm": "SVM",
    "Tdnn": "TDNN",
    "Tech": "Technology",
    "Tub": "TUB",
    "Uwave": "uWave",
    "Vpl": "VPL",
    "Vr": "Virtual Reality",
    "Wifi": "WiFi",
#     "Naive Bayes": "Naïve-Bayes", # Matplotlib doesn't understand 'ï'
}

rm_acronyms = {
    "CNN": "Convolutional Neural Network",
    "EMG": "Electromyography",
    "FFNN": "Feed-Forward Neural Network",
    "GMM": "Gaussian Mixture Model",
    "GRU": "Gated Recurrent Unit",
    "HMM": "Hidden Markov Model",
    "IMU": "Inertial Measurement Unit",
    "KNN": "K-Nearest Neighbors",
    "LSTM": "Long Short-Term Memory",
    "NN": "Neural Network",
    "PCA": "Principal Component Analysis",
    "RGB": "Red Green Blue",
    "RGBD": "Red Green Blue Depth",
    "RNN": "Recurrent Neural Network",
    "SOM": "Self-Organizing Map",
    "SVD": "Singular Value Decomposition",
    "SVM": "Support Vector Machine",
    "TDNN": "Time-Delay Neural Network",
    "TUB": "Technical University of Berlin",
}

def prettify_text(text, remove_acronyms=False):
    text = text.replace('-', ' ').title()
    for k, v in replacements.items():
        text = re.sub(r'\b' + k + r'\b', v, text, 0)
    if remove_acronyms:
        for k, v in rm_acronyms.items():
            text = re.sub(r'\b' + k + r'\b', v, text, 0)
    return text
    
def prettify_ax(ax, remove_acronyms=False):
    ax.grid()
    ax.set_xlabel(prettify_text(ax.get_xlabel(), remove_acronyms=remove_acronyms))
    ax.set_ylabel(prettify_text(ax.get_ylabel(), remove_acronyms=remove_acronyms))

#     ax.set_xticks(ax.get_xticks())
#     ax.set_xticklabels([
#         prettify_text(label.get_text())
#         for label
#         in ax.get_xticklabels()
#     ])
    ax.set_yticks(ax.get_yticks())
    ax.set_yticklabels([
        prettify_text(label.get_text(), remove_acronyms=remove_acronyms)
        for label
        in ax.get_yticklabels()
    ])

    legend = ax.get_legend()
    if legend:
        for txt in legend.get_texts():
            txt.set_text(prettify_text(
                txt.get_text(), 
                remove_acronyms=remove_acronyms
            ))

        legend.set_title(prettify_text(
            legend.get_title().get_text(), 
            remove_acronyms=remove_acronyms
        ))

    return ax

def prettify_xticks(ax, remove_acronyms=False):
    ax.set_xticks(ax.get_xticks())
    ax.set_xticklabels([
        prettify_text(label.get_text(), remove_acronyms=remove_acronyms)
        for label
        in ax.get_xticklabels()
    ])

def prettify_yticks(ax, remove_acronyms=False):
    ax.set_yticks(ax.get_yticks())
    ax.set_yticklabels([
        prettify_text(label.get_text(), remove_acronyms=remove_acronyms)
        for label
        in ax.get_yticklabels()
    ])

def dedup_prefix(df, prefix):
    """Taken in a dataframe and a prefix column that contains values like `abc+123+efg.
    Split those values up so they take multiple rows, containing `abc`, `123`, `efg`."""
    assert prefix in df.columns, f'prefix `{prefix}` is not in df.columns:\n{df.columns.to_list()}'
    list_of_series = []
    for i, row in df.iterrows():
        if pd.notna(row[prefix]) and '+' in row[prefix]:
            for item in row[prefix].split('+'):
                tmp_row = row.copy()
                tmp_row[prefix] = item
                list_of_series.append(tmp_row)
        else:
            list_of_series.append(row)
    return pd.DataFrame(list_of_series).reset_index(drop=True)

def fmt_legend(ax=None, handles_labels=None, **kwargs):
    if ax is None:
        ax = plt.gca()
    if handles_labels is None:
        handles, labels = ax.get_legend_handles_labels()
    else:
        handles, labels = handles_labels
    ax.legend(
        handles=handles,
        labels=labels,
        borderpad=0.5,
        labelspacing=0.1,
        handlelength=0.1,
        **kwargs,
    )
    return ax

def add_grid(ax=None, **kwargs):
    if ax is None:
        ax = plt.gca()
    ax.grid(
        'both',
        **(
            dict(
                alpha=0.25,
                lw=1,
            ) | kwargs
        )
    )
    ax.set_axisbelow(True)
    return ax

def add_cbar(
    fig, ax, data,
    label=None,
    cmap='Spectral',
    vmin=None,
    vmax=None,
) -> None:
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='2.5%', pad=0.05)

    norm = mpl.colors.Normalize(
        vmin=vmin if vmin is not None else data.min(),
        vmax=vmax if vmax is not None else data.max(),
    )

    cb = fig.colorbar(
        mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
        cax=cax,
        orientation='vertical',
        label=label
    )
    cb.outline.set_visible(False)

stripplot_kwargs = dict(
    s=2.5,
    linewidth=.1,
    alpha=0.75,
    edgecolor='#000a',
)

scatterplot_kwargs = dict(
    s=5,
    edgecolor='#000a',
    linewidth=0.1,
    alpha=0.75,
)

## Read in the data

In [ ]:
# Read in the data
date_cols = ['Date', 'Publication Year', 'Date Added', 'Date Modified', 'Access Date']
df = pd.read_csv(
    './src/zotero.csv', 
    parse_dates=date_cols
)
df['Date'] = pd.to_datetime(df['Date'], format='ISO8601')

def get_citations(x):
    if x is None or pd.isna(x):
        return None
    RE = r'(\d+) citations'
    match = re.search(RE, x, flags=re.IGNORECASE)
    return None if match is None else int(match.groups()[0])

# Extract citations
df['citations'] = df['Extra'].apply(get_citations)

# Extract nicely formatted tags as a list of key:value objects
df['tags'] = df['Manual Tags'].str.split('; ').apply(lambda x: x if type(x) is list or pd.notna(x) else [])

# # https://stackoverflow.com/a/952952/14555505
uniq_tags = sorted(list(set([item for sublist in df['tags'] for item in sublist])))

# Make a helper function to check if tag(s) is/are in a series
def has_tag(tag, series):
    if type(tag) is list:
        return series.apply(lambda x: any(t in x for t in tag))
    else:
        return series.apply(lambda x: tag in x)

# Make a helper function to extract the references from the Extras field
def extract_references(extra):
    if pd.isna(extra):
        return []
    regex = r'--- DOIs_of_references: (\[.*\]) ---'
    match = re.search(regex, extra)
    if match is None:
        return []
    dois_str = match.groups()[0]
    return sorted([l[1:-1] for l in dois_str[1:-1].split(',')])

df['references'] = df['Extra'].apply(extract_references)

# Get a shortcut column for the year
df['year'] = df['Date'].dt.year

In [ ]:
# Organise the data

# Add columns to the dataset, one for each prefix
prefixes = sorted(set([t.split(':')[0] for t in uniq_tags if ':' in t]))

# Make a column for all `prefix`:`value` type tags
def make_prefix_value(tags, prefix, sep=':'):
    s = '+'.join(sorted([tag.replace(prefix + sep, '') for tag in tags if (prefix+sep) in tag]))
    return s if s else np.NaN

for prefix in prefixes:
    df[prefix] = df['tags'].apply(lambda tags: make_prefix_value(tags, prefix))

# Attempt to infer better dtypes for object columns.
df = df.infer_objects()

# Fidelity over time

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
deduped = dedup_prefix(dedup_prefix(df, 'fidelity'), 'based-on')
deduped = deduped[
    deduped['type'].isin(['paper', 'msc']) &
    (deduped['based-on'] == 'gloves') & 
    deduped['fidelity'].notna()
]
deduped['citations'] = deduped['citations'].fillna(0)
sns.swarmplot(
    data=deduped.sort_values('Date'), 
    x="Date", 
    y="fidelity",
    order=['finger', 'hand', 'arm'],
    hue_norm=(0, deduped['citations'].max()),
    hue='citations',
    legend=False,
    linewidth=0.5,
    edgecolor='#000a',
    palette='Spectral',
    size=4, 
    ax=ax
)
# watermark(ax)
ax = prettify_ax(ax)

add_grid(ax)
add_cbar(
    fig, ax, deduped['citations'], label='Citations'
)
# fmt_legend(ax, title='Citations')
ax.set(
    xlabel='Year',
    ylabel='Fidelity',
    title='Gesture Fidelity',
#     xticks=ax.get_xticks()[::2],
#     xticklabels=ax.get_xticklabels()[::2],
)
prettify_yticks(ax)

nfinger = (deduped['fidelity'] == 'finger').sum()
nhand = (deduped['fidelity'] == 'hand').sum()
narm = (deduped['fidelity'] == 'arm').sum()
ax.set_yticklabels([
    f'Finger ({nfinger})', 
    f'Hand ({nhand})',
    f'Arm ({narm})',
])

plt.tight_layout()
plt.savefig(f'./{ORAL_DIR}/03_fidelity_for_gloves.pdf', bbox_inches='tight')
# plt.show()

In [ ]:
deduped = dedup_prefix(dedup_prefix(df, 'fidelity'), 'based-on')
deduped = deduped[
    deduped['type'].isin(['paper', 'msc']) &
    (deduped['based-on'] == 'gloves') & 
    deduped['fidelity'].notna()
]
deduped['citations'] = deduped['citations'].fillna(0)
deduped['fidelity'].value_counts()

# Segmentation

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
deduped = dedup_prefix(dedup_prefix(df, 'segmentation'), 'based-on')

# deduped = dedup_prefix(dedup_prefix(df, 'hardware'), 'based-on')
deduped = deduped[
    deduped['type'].isin(['paper', 'msc']) &
    deduped['segmentation'].isin(['implicit', 'explicit']) &
    (deduped['based-on'] == 'gloves')
]

deduped['citations'] = deduped['citations'].fillna(0)
sns.swarmplot(
    data=deduped.sort_values('Date'), 
    x="Date", 
    y="segmentation",
#     order=['finger', 'hand', 'arm+hand', 'arm'],
    hue_norm=(0, deduped['citations'].max()),
    hue='citations',
    legend=False,
    linewidth=0.5,
    edgecolor='#000a',
    palette='Spectral',
    size=4, 
    ax=ax
)
# watermark(ax)
ax = prettify_ax(ax)
add_grid(ax)
add_cbar(
    fig, ax, deduped['citations'], label='Citations'
)
ax.set(
    xlabel='Year',
    ylabel='Segmentation',
    title='Gesture Segmentation',
)
prettify_yticks(ax)
nexplicit = (deduped['segmentation'] == 'explicit').sum()
nimplicit = (deduped['segmentation'] == 'implicit').sum()
ax.set_yticklabels([f'Implicit ({nimplicit})', f'Explicit ({nexplicit})'])
plt.tight_layout()
plt.savefig(f'./{ORAL_DIR}/03_segmentation_for_gloves.pdf', bbox_inches='tight')
plt.show()

In [ ]:
(deduped['segmentation'] == 'implicit').sum()

# Static vs Dynamic Movement

In [ ]:
# fig, axs = plt.subplots(2, 1, figsize=(8, 5))
data = dedup_prefix(dedup_prefix(df, 'movement'), 'based-on')

sns.swarmplot(
    data=data,
    x="Date", 
    hue="movement",
    y='based-on',
    size=4,
#     legend=False,
)
plt.tight_layout()

# Hardware used for glove-based systems

In [ ]:
fig, ax = plt.subplots(figsize=(WIDTH, 7))

deduped = dedup_prefix(dedup_prefix(df, 'hardware'), 'based-on')
deduped = deduped[
    (deduped.type == 'paper') &
    (deduped['based-on'] == 'gloves') & 
    deduped['hardware'].notna()
]
deduped['citations'] = deduped['citations'].fillna(0)
sns.swarmplot(
    data=deduped.sort_values('Date'), 
    x="Date", 
    y="hardware",
    hue_norm=(0, deduped['citations'].max()),
    hue='citations',
    legend=False,
    linewidth=0.5,
    edgecolor='#000a',
    palette='Spectral',
    size=4, 
    ax=ax
)
# watermark(ax)
# ax = prettify_ax(ax)

add_grid(ax)
add_cbar(
    fig, ax, deduped['citations'], label='Citations'
)
# fmt_legend(ax, title='Citations')
ax.set(
    xlabel='Year',
    ylabel='Hardware used',
    title='Hardware used for glove-based systems',
    xticks=ax.get_xticks()[::2],
    xticklabels=ax.get_xticklabels()[::2],
)
prettify_yticks(ax)

# plt.savefig('./src/imgs/graphs/03_hardware_for_gloves.pdf', bbox_inches='tight')
# plt.tight_layout()
# plt.show()

# Tech used for glove-based systems

In [ ]:
fig, ax = plt.subplots(figsize=(WIDTH, WIDTH))

deduped = dedup_prefix(dedup_prefix(df, 'tech'), 'based-on')

deduped = deduped[
    (deduped.type == 'paper') &
    (deduped['based-on'] == 'gloves') &
    (~deduped['tech'].isin(['rgb'])) &
    (deduped['tech'].notna())
].sort_values('Date')
deduped['citations'] = deduped['citations'].fillna(0)

sns.swarmplot(
    data=deduped, 
    x="Date", 
    y="tech",
    hue='citations',
    hue_norm=(0, deduped['citations'].max()),
    palette='Spectral',
    edgecolor='#000',
    linewidth=0.25,
    legend=False,
    size=4, 
    ax=ax
)
add_grid(ax)
add_cbar(
    fig, ax, deduped['citations'], label='Citations'
)
# fmt_legend(ax, title='Citations')
ax.set(
    xlabel='Year',
    ylabel='Technology used',
    title='Glove-based technology over time',
    xticks=ax.get_xticks()[::2],
    xticklabels=ax.get_xticklabels()[::2],
)
prettify_yticks(ax)

plt.savefig('./src/imgs/graphs/03_tech_for_gloves.pdf', bbox_inches='tight')

## Glove-based tech over time w/o outlier

In [ ]:
fig, ax = plt.subplots(figsize=(WIDTH, WIDTH))

deduped = dedup_prefix(dedup_prefix(df, 'tech'), 'based-on')

deduped = deduped[
    (deduped.type == 'paper') &
    (deduped['based-on'] == 'gloves') &
    (~deduped['tech'].isin(['rgb'])) &
    (deduped['tech'].notna())
].sort_values('Date')

deduped['citations'] = deduped['citations'].fillna(0)
# Remove the one outlier that's a highly-cited medical paper
deduped = deduped[deduped['citations'] < 1000]


sns.swarmplot(
    data=deduped, 
    x="Date", 
    y="tech",
    hue='citations',
    hue_norm=(0, deduped['citations'].max()),
    palette='Spectral',
    edgecolor='#000',
    linewidth=0.25,
    legend=False,
    size=4, 
    ax=ax
)
add_grid(ax)
add_cbar(
    fig, ax, deduped['citations'], label='Citations'
)
# fmt_legend(ax, title='Citations')
ax.set(
    xlabel='Year',
    ylabel='Technology used',
    title='Glove-based Technology Over Time',
)
prettify_yticks(ax,remove_acronyms=True)

plt.savefig(f'./{ORAL_DIR}/03_tech_for_gloves.pdf', bbox_inches='tight')

In [ ]:
%%script false --no-raise-error # This cell is unused
fig, ax = plt.subplots(figsize=(A4_WIDTH_INCHES,16))

data = dedup_prefix(dedup_prefix(df, 'app'), 'based-on').sort_values('Date')
data = data[
    data['app'].notna()
]

sns.swarmplot(
    data=data, 
    x="Date", 
    y="app",
    hue='based-on',
    size=4, 
    ax=ax
)
watermark(ax)
ax = prettify_ax(ax)

plt.title('Applications of Gesture Recognition')
# plt.savefig('./src/imgs/graphs/03_applications.pdf', bbox_inches='tight')

# Applications of vision/wifi gesture detection

In [ ]:
fig, ax = plt.subplots(figsize=(WIDTH, WIDTH*1.25))

data = dedup_prefix(dedup_prefix(df, 'app'), 'based-on').sort_values('app')
data = data[
    (data['based-on'].isin(['wifi', 'vision'])) & 
    (data['app'].notna()) & 
    (data['app'].apply(lambda x: '-sl' not in str(x)))
].sort_values('Date')
# order = data.groupby('app').size().sort_values(ascending=False).index

sns.swarmplot(
    data=data, 
    x="Date", 
    y="app",
    hue='based-on',
#     order=order,
    size=4, 
    ax=ax
)
watermark(ax)
ax = prettify_ax(ax)

plt.title('Applications of vision/WiFi-based\ngesture recognition')
plt.savefig('./src/imgs/graphs/03_wifi_vision_applications.pdf', bbox_inches='tight')

# Sign-language applications

In [ ]:
%%script false --no-raise-error # This cell is unused
fig, ax = plt.subplots(figsize=(WIDTH, 5))
data = dedup_prefix(dedup_prefix(df, 'app'), 'based-on').sort_values('app')
data = data[data.app.apply(lambda x: '-sl' in str(x))]
sns.swarmplot(
    data=data, 
    x="Date", 
    y="app",
    hue='based-on',
    size=4, 
    ax=ax
)

add_grid(ax)
ax.set(
    xlabel='Year',
    ylabel='Sign Language',
    title='Sign language applications of\ngesture recognition',
    xticks=ax.get_xticks()[::2],
    xticklabels=ax.get_xticklabels()[::2],
    yticks=ax.get_yticks(),
    yticklabels=[
        t.get_text().replace('-', ' ').replace(' sl', '').title()
        for t 
        in ax.get_yticklabels()
    ]
)
fmt_legend(ax)

plt.savefig('./src/imgs/graphs/03_sl_applications.pdf', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(WIDTH, 8))
data = dedup_prefix(dedup_prefix(df, 'model'), 'based-on').sort_values('Date')

value_counts = data.value_counts('model')
gt_1_use = value_counts[value_counts > 1].index
data = data[
    (data['model'].isin(gt_1_use)) &
    (data['based-on'] != 'gloves') &
    (data['based-on'].notna())
]
print((value_counts <= 1).sum())
# order = data.groupby('model').size().sort_values(ascending=False).index
sns.swarmplot(
    data=data.sort_values('Date'),
    x="Date",
    y="model",
    hue='based-on',
#     order=order,
    size=4,
    ax=ax
)
watermark(ax)
ax = prettify_ax(ax)

plt.title('Models Used for Non-glove-based\nGesture Recognition')
# plt.tight_layout()
plt.savefig('./src/imgs/graphs/03_models_no_gloves.pdf', bbox_inches='tight')

# Classificaiton models used

## Glove-based

In [ ]:
fig, ax = plt.subplots(figsize=(WIDTH, WIDTH))
data = dedup_prefix(dedup_prefix(df, 'model'), 'based-on').sort_values('Date')

data = data[
    (data.type == 'paper') & 
    (data['based-on'] == 'gloves')
]
print("excluding models: ", (data.value_counts('model') == 1).sum())
value_counts = data.value_counts('model')
gt_1_use = value_counts[value_counts > 1].index
data = data[
    (data['model'].isin(gt_1_use))
].sort_values("Date")

data['citations'] = data['citations'].fillna(0)

# Remove the one outlier that's a highly-cited medical paper
mask = data['citations'] > 1000
assert (mask).sum() == 1
data = data[~mask]
sns.swarmplot(
    data=data,
    x="Date",
    y="model",
    hue='citations',
    hue_norm=(0, data['citations'].max()),
    palette='Spectral',
    edgecolor='#000a',
    linewidth=0.5,
    legend=False,
    size=4,
    ax=ax
)
prettify_yticks(ax, remove_acronyms=True)
add_grid(ax)
add_cbar(
    fig, ax, data['citations'], label='Citations'
)

ax.set(
    title='Algorithms used for glove-based\ngesture recognition',
    xticks=ax.get_xticks()[::2],
    xticklabels=ax.get_xticklabels()[::2],
    xlabel='Year',
    ylabel='Algorithm used',
)

plt.savefig(f'./{ORAL_DIR}/03_models_glove_based.pdf', bbox_inches='tight')
plt.tight_layout()
plt.show()

In [ ]:
data.sort_values('citations', ascending=False)[['Author', 'Title', 'citations', 'Date']]

## Non-glove-based

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(2*WIDTH, 2*WIDTH))
data = dedup_prefix(dedup_prefix(df, 'model'), 'based-on').sort_values('Date')

data = data[(data.type == 'paper') & (data['based-on'] != 'gloves')]

value_counts = data.value_counts('model')
gt_1_use = value_counts[value_counts > 1].index
data = data[
    (data['model'].isin(gt_1_use))
].sort_values("Date")

order = data['model'].unique()
hue_norm = (data['citations'].min(), data['citations'].max())
for i, based_on in enumerate(['vision', 'wifi']):
    sns.swarmplot(
        data=data[data['based-on'] == based_on],
        x="Date",
        y="model",
        order=order,
        hue='citations',
        edgecolor='#000',
        linewidth=0.5,
        hue_norm=hue_norm,
#         xlim=(data['Date'].min(), data['Date'].max()),
        size=4,
        palette='Spectral',
        ax=axs[i],
        legend=False,
    )
    prettify_yticks(axs[i])
    add_grid(axs[i])
    print(axs[i].get_xlim())
    axs[i].set(
        xlabel='Year',
        ylabel='Model used',
    )
    if i == 0:    
        axs[i].set(
            xticklabels=[],
            xlabel=None,
        )
    add_cbar(
        fig, axs[i], label='Citations', data=data['citations'],
    )
plt.savefig('./src/imgs/graphs/03_models_no_gloves_other.pdf', bbox_inches='tight')
plt.tight_layout()
plt.show()

In [ ]:
# HMMs vs CNNs/NNs
fig, ax = plt.subplots(figsize=(A4_WIDTH_INCHES, 4))
data = dedup_prefix(dedup_prefix(df, 'model'), 'based-on').sort_values('Date')
data = data[
    (data.model.isin(['nn', 'cnn', 'hmm'])) &
    (data['based-on'].isin(['vision', 'wifi']))
]

# order = data.groupby('model').size().sort_values(ascending=False).index
sns.swarmplot(
    data=data,
    x="Date",
    y="model",
    hue='based-on',
#     order=order,
    size=4,
    ax=ax
)
watermark(ax)
ax = prettify_ax(ax)

plt.title('HMMs and CNNs for Vision/WiFi-based Gesture Recognition')
# plt.savefig('./src/imgs/graphs/03_models_hmm_vs_cnn.pdf', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(A4_WIDTH_INCHES,5))
data = dedup_prefix(dedup_prefix(dedup_prefix(df, 'model'), 'based-on'), 'participants').sort_values('model')

data['participants'] = (pd.to_numeric(data['participants']))

data = data[
    data['model'].notna()
]

sns.scatterplot(
    data=data,
    y="participants",
    x="Date",
    hue='based-on',
    ax=ax,
)

# watermark(ax)
# ax = prettify_ax(ax)

# ax.set_ylabel('Number of Participants')

# ax.set_xticks(ax.get_xticks())
# ax.set_xticklabels(np.round(np.power(10, ax.get_xticks()), 0))

# plt.title('Model Used vs Number of Participants')
# plt.savefig('./src/imgs/graphs/03_models_vs_nclasses.pdf', bbox_inches='tight')

# Number of classes vs ...

## Number of classes vs date

In [ ]:
fig, ax = plt.subplots(figsize=(WIDTH, 0.75*WIDTH))
data = dedup_prefix(dedup_prefix(dedup_prefix(df, 'model'), 'based-on'), 'classes').sort_values('Date')

data = data[
    (data.type.isin(['paper', 'msc']))
    & (data['classes'].notna())
]
data['classes'] = np.log10(pd.to_numeric(data['classes']))
# value_counts = data.value_counts('model')
# gt_1_use = value_counts[value_counts > 1].index
# data = data[
#     (data['model'].isin(gt_1_use)) &
#     data['model'].notna() &
#     data['classes'].notna()
# ]

sns.scatterplot(
    data=data,
    y="classes",
    x="Date",
    hue='based-on',
    **(scatterplot_kwargs | {
        's': 10
    }),
#     s=4,
    ax=ax,
)

watermark(ax)
# ax = prettify_ax(ax)

ax.set(
    ylabel='Number of Classes',
    title='Number of Classes over Time'
)

add_grid(ax)
yticks =  ax.get_yticks()
yticks = np.log10([1, 5, 10, 26, 50, 100, 500, 1000, 5000])
handles, labels = ax.get_legend_handles_labels()
ax.legend(
    handles=handles,
    labels=['Gloves', 'Vision', 'WiFi'],
    borderpad=0.5,
    labelspacing=0.1,
    handlelength=0.1,
    title='Based-on',
)
print(yticks)
ax.set_yticks(yticks)
ax.set_yticklabels((np.round(np.power(10, yticks), 0)).astype(int))


# plt.title('Model Used vs Number of Classes')
# plt.savefig('./src/imgs/graphs/03_date_vs_nclasses.pdf', bbox_inches='tight')

## Nclasses vs date (hue=citations)

In [ ]:
fig, ax = plt.subplots(figsize=(WIDTH, 0.75*WIDTH))
data = dedup_prefix(dedup_prefix(df, 'based-on'), 'classes').sort_values('Date')

data = data[
    (data.type.isin(['paper', 'msc']))
    & (data['classes'].notna())
    & (data['based-on'] == 'gloves')
]
data['citations'] = data['citations'].fillna(0)
# Remove the one biotech paper that's an outlier
data = data[data['citations'] < 1000]
data['classes'] = np.log10(pd.to_numeric(data['classes']))
# data['classes'] = (pd.to_numeric(data['classes']))
# data = data[data['classes'] < 2500]
sns.scatterplot(
    data=data,
    y="classes",
    x="Date",
    hue_norm=(0, data['citations'].max()),
    hue='citations',
    legend=False,
    linewidth=0.5,
    edgecolor='#000a',
    palette='Spectral',
    size=4, 
    ax=ax,
)

yticks =  ax.get_yticks()
yticks = np.log10([1, 2, 5, 10, 26, 50, 100, 200, 500, 1000, 2000, 5000])
# yticks = np.log10([10, 20, 40, 80, 100, 200, 400, 800, 1000, 2000, 4000, 8000])
ax.set_yticks(yticks)
ax.set_yticklabels((np.round(np.power(10, yticks), 0)).astype(int))
prettify_yticks(ax)
ax = prettify_ax(ax)
add_grid(ax)
add_cbar(
    fig, ax, data['citations'], label='Citations'
)
ax.set(
    xlabel='Year',
    ylabel='Number of Classes',
    title='Number of Classes over Time',
)
plt.tight_layout()
plt.savefig(f'./{ORAL_DIR}/03_date_vs_nclasses.pdf', bbox_inches='tight')
plt.show()

In [ ]:
data.sort_values('classes', ascending=False)[['classes', 'Title']].assign(**{
    'classes': lambda ddf: np.power(10, ddf['classes'])
}).head(10)

## Number of classes vs model used

In [ ]:
fig, ax = plt.subplots(figsize=(A4_WIDTH_INCHES,14))
data = dedup_prefix(dedup_prefix(dedup_prefix(df, 'model'), 'based-on'), 'classes').sort_values('model')

data = data[data.type.isin(['paper', 'msc'])]


data['classes'] = np.log10(pd.to_numeric(data['classes']))

value_counts = data.value_counts('model')
gt_1_use = value_counts[value_counts > 1].index
data = data[
    (data['model'].isin(gt_1_use)) &
    data['model'].notna() &
    data['classes'].notna()
]

sns.swarmplot(
    data=data,
    y="model",
    x="classes",
    hue='based-on',
    size=4,
    ax=ax,
)

watermark(ax)
ax = prettify_ax(ax)

ax.set_xlabel('Number of Classes ($\log_{10}$ scale)')

ax.set_xticks(ax.get_xticks())
ax.set_xticklabels((np.round(np.power(10, ax.get_xticks()), 0)).astype(int))


plt.title('Model Used vs Number of Classes')
plt.savefig('./src/imgs/graphs/03_models_vs_nclasses.pdf', bbox_inches='tight')

## Num classes vs fidelity

In [ ]:
fig, ax = plt.subplots(figsize=(WIDTH, 0.75*WIDTH))
data = dedup_prefix(dedup_prefix(dedup_prefix(df, 'fidelity'), 'based-on'), 'hardware').sort_values('Date')

data = data[
    (data.type.isin(['paper', 'msc'])) &
    (data['based-on'] == 'gloves') &
    (data['fidelity'].notna())
]

# Pivot the dataframe into a heatmap-style count plot
heatmap = data[['fidelity', 'hardware']].reset_index().pivot_table(
    values='index', 
    columns='fidelity', 
    index='hardware', 
    aggfunc='count', 
    fill_value=0
)
# Reorder columns and rows:
heatmap = heatmap[['finger', 'hand', 'arm']]
uniq_hardware = data['hardware'].value_counts().index
heatmap = heatmap.reindex(uniq_hardware)

# data['classes'] = np.log10(pd.to_numeric(data['classes']))
# value_counts = data.value_counts('model')
# gt_1_use = value_counts[value_counts > 1].index
# data = data[
#     (data['model'].isin(gt_1_use)) &
#     data['model'].notna() &
#     data['classes'].notna()
# ]

sns.heatmap(
    data=heatmap,
    ax=ax,
    annot=True,
    cmap='Spectral',
    mask=(heatmap == 0)
)

watermark(ax)
ax = prettify_ax(ax)

# ax.set(
#     ylabel='Number of Classes',
#     title='Number of Classes over Time'
# )

add_grid(ax)
# xticks =  ax.get_xticks()
# xticks = np.log10([1, 5, 10, 26, 50, 100, 500, 1000, 5000])
# handles, labels = ax.get_legend_handles_labels()
# ax.legend(
#     handles=handles,
#     labels=['Gloves', 'Vision', 'WiFi'],
#     borderpad=0.5,
#     labelspacing=0.1,
#     handlelength=0.1,
#     title='Based-on',
# )
# print(yticks)
# ax.set_xticks(xticks)
# ax.set_xticklabels((np.round(np.power(10, xticks), 0)).astype(int))


# plt.title('Model Used vs Number of Classes')
# plt.savefig('./src/imgs/graphs/03_fidelity_vs_nclasses.pdf', bbox_inches='tight')
plt.show()

In [ ]:
data.loc[
    data['classes'] > np.log10(50),
#     ['year', 'Title', 'Author', 'Url', 'classes', 'citations']
    'Title'
].tolist()#.assign(**{"classes": lambda ddf: np.power(10, ddf['classes'])})

# Glove/vision/wifi based systems over time

In [ ]:
fig, ax = plt.subplots(figsize=(WIDTH, 0.75*WIDTH))
deduped = dedup_prefix(dedup_prefix(df, 'based-on'), 'type')
deduped = deduped[
    deduped.type.isin(['paper', 'survey', 'msc', 'dataset'])
]

sns.swarmplot(
    data=deduped,
    x="Date", 
    y="based-on",
    order=deduped['based-on'].value_counts().index,
    hue='type',
    edgecolor='#000',
    linewidth=0.25,
    size=4, 
    ax=ax,
)
# watermark(ax)
# ax = prettify_ax(ax)

add_grid(ax)
# handles_labels = ax.get_legend_handles_labels()
handles_labels = (
    ax.get_legend_handles_labels()[0],
    [l.title() for l in ax.get_legend_handles_labels()[1]]
)

fmt_legend(ax, handles_labels=handles_labels)
ax.set(
    xlabel='Year',
    ylabel='Based-on',
    title='Different systems over time',
#     xticks=ax.get_xticks()[::2],
#     xticklabels=ax.get_xticklabels()[::2],
)
prettify_yticks(ax)

# plt.title('Trend of Different Systems over Time')
plt.savefig('./src/imgs/graphs/03_based_on_over_time.pdf', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(A4_WIDTH_INCHES,4))

data = dedup_prefix(dedup_prefix(df, 'fidelity'), 'based-on')

data = data[
    data['fidelity'].notna()
]
sns.swarmplot(
    data=data, 
    x="Date", 
    y="fidelity",
    hue='based-on',
    size=8,
    ax=ax
)
watermark(ax)    
ax = prettify_ax(ax)


plt.title('Fidelity of over Time')

In [ ]:
# TODO: Save as LaTeX
latex = df.loc[
    (df.type == 'dataset'),
    ['Title', 'Author', 'Date', 'dataset', 'participants', 'observations', 'classes', 'tech', 'hardware']
]

with open('./src/tables/03_datasets.generated.tex', 'w') as f:
    latex.to_latex(
        f,
        caption='caption',
        label='tab:',
        index=False,
        bold_rows=True,
        longtable=True,
        na_rep='-',
        formatters={
            "Date": lambda d: str(d)[:-9],
        }
    )